# SageMaker Training Job for Dating Profile Matcher

이 노트북은 SageMaker AI Studio에서 Qwen3-VL 기반 프로필 매칭 모델을 학습하는 방법을 보여줍니다.

## 1. 환경 설정

In [ ]:
# 필수 라이브러리 설치
!pip install -q sagemaker boto3 transformers qwen-vl-utils

In [ ]:
import sagemaker
import boto3
from sagemaker.pytorch import PyTorch
from sagemaker.inputs import TrainingInput
from datetime import datetime
import os

# SageMaker 세션
sess = sagemaker.Session()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

print(f"리전: {region}")
print(f"역할: {role}")
print(f"세션 버킷: {sess.default_bucket()}")

## 2. 데이터 S3 업로드

In [ ]:
# S3 경로 설정
bucket = sess.default_bucket()
prefix = 'dating-matcher'

# 로컬 데이터 경로
local_data_dir = os.path.expanduser('~/SageMaker/dating-matcher/data/processed')

# S3에 업로드
s3_data_uri = sess.upload_data(
    path=local_data_dir,
    bucket=bucket,
    key_prefix=f"{prefix}/data/processed"
)

print(f"데이터가 업로드되었습니다: {s3_data_uri}")

## 3. 학습 작업 설정

In [ ]:
# 하이퍼파라미터 설정
hyperparameters = {
    'config': 'configs/config_sagemaker.yaml',
    'learning-rate': 5e-5,
    'batch-size': 16,
    'num-epochs': 30,
    'embedding-dim': 512
}

# 메트릭 정의
metric_definitions = [
    {'Name': 'train:loss', 'Regex': 'Train Loss: ([0-9\\.]+)'},
    {'Name': 'validation:loss', 'Regex': 'Val Loss: ([0-9\\.]+)'},
    {'Name': 'learning_rate', 'Regex': 'LR: ([0-9\\.]+)'}
]

print("하이퍼파라미터:")
for key, value in hyperparameters.items():
    print(f"  {key}: {value}")

In [ ]:
# PyTorch Estimator 생성
estimator = PyTorch(
    entry_point='train_sagemaker.py',
    source_dir='../src/training',
    dependencies=['../src', '../configs'],
    role=role,
    instance_type='ml.g5.xlarge',  # GPU 인스턴스
    instance_count=1,
    framework_version='2.1.0',
    py_version='py310',
    hyperparameters=hyperparameters,
    metric_definitions=metric_definitions,
    
    # 체크포인트 저장
    checkpoint_s3_uri=f"s3://{bucket}/{prefix}/checkpoints",
    checkpoint_local_path='/opt/ml/checkpoints',
    
    # Spot 인스턴스 사용 (비용 절감)
    use_spot_instances=True,
    max_run=24*60*60,  # 24시간
    max_wait=24*60*60,
    
    # 볼륨 크기
    volume_size=50,
    
    # 환경 변수
    environment={
        'TRANSFORMERS_CACHE': '/tmp/transformers_cache',
        'HF_HOME': '/tmp/huggingface'
    }
)

print("Estimator 생성 완료!")

## 4. 학습 작업 시작

In [ ]:
# 학습 시작
print("학습 작업을 시작합니다...")
print(f"데이터 경로: {s3_data_uri}")

estimator.fit(
    inputs={'training': s3_data_uri},
    wait=False  # False로 설정하면 비동기 실행
)

print(f"\n학습 작업 이름: {estimator.latest_training_job.name}")
print("\nAWS Console에서 진행상황을 확인하세요:")
print(f"https://{region}.console.aws.amazon.com/sagemaker/home?region={region}#/jobs/{estimator.latest_training_job.name}")

## 5. 학습 모니터링

In [ ]:
# 학습 작업 상태 확인
import time

def check_training_status(estimator):
    """학습 작업 상태 확인"""
    job_name = estimator.latest_training_job.name
    client = boto3.client('sagemaker')
    
    response = client.describe_training_job(TrainingJobName=job_name)
    
    status = response['TrainingJobStatus']
    print(f"상태: {status}")
    
    if status == 'InProgress':
        print(f"진행 시간: {response.get('TrainingStartTime')}")
        if 'SecondaryStatusTransitions' in response:
            for transition in response['SecondaryStatusTransitions'][-3:]:
                print(f"  {transition['Status']}: {transition.get('StatusMessage', 'N/A')}")
    
    elif status == 'Completed':
        print(f"✓ 학습 완료!")
        print(f"모델 위치: {response['ModelArtifacts']['S3ModelArtifacts']}")
    
    elif status == 'Failed':
        print(f"✗ 학습 실패: {response.get('FailureReason', 'Unknown')}")
    
    return status

# 상태 확인
check_training_status(estimator)

In [ ]:
# 학습 로그 확인 (실시간)
# estimator.logs()  # 주의: 이 셀을 실행하면 로그가 스트리밍됩니다

## 6. 학습된 모델 다운로드

In [ ]:
# 학습 완료 대기
# estimator.latest_training_job.wait(logs='None')

# 모델 아티팩트 다운로드
import tarfile

model_data = estimator.model_data
print(f"모델 위치: {model_data}")

# 다운로드
local_model_dir = os.path.expanduser('~/SageMaker/dating-matcher/models/sagemaker')
os.makedirs(local_model_dir, exist_ok=True)

!aws s3 cp {model_data} {local_model_dir}/model.tar.gz

# 압축 해제
with tarfile.open(f"{local_model_dir}/model.tar.gz") as tar:
    tar.extractall(path=local_model_dir)

print(f"모델이 다운로드되었습니다: {local_model_dir}")

## 7. 모델 배포 (선택사항)

In [ ]:
# SageMaker 엔드포인트로 배포
# 주의: 엔드포인트는 시간당 비용이 발생합니다!

endpoint_name = f"dating-matcher-{datetime.now().strftime('%Y%m%d-%H%M%S')}"

print(f"엔드포인트 배포 시작: {endpoint_name}")
print("이 작업은 5-10분 정도 소요됩니다...")

# predictor = estimator.deploy(
#     initial_instance_count=1,
#     instance_type='ml.g5.xlarge',
#     endpoint_name=endpoint_name
# )

# print(f"✓ 엔드포인트 배포 완료: {endpoint_name}")

In [ ]:
# 엔드포인트 삭제 (비용 절감)
# predictor.delete_endpoint()
# print("엔드포인트가 삭제되었습니다.")

## 8. 하이퍼파라미터 튜닝 (선택사항)

In [ ]:
from sagemaker.tuner import (
    HyperparameterTuner,
    ContinuousParameter,
    CategoricalParameter
)

# 하이퍼파라미터 범위 정의
hyperparameter_ranges = {
    'learning-rate': ContinuousParameter(1e-5, 1e-4),
    'batch-size': CategoricalParameter([8, 16, 32]),
}

# 최적화 목표
objective_metric_name = 'validation:loss'
objective_type = 'Minimize'

# Tuner 생성
tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    max_jobs=10,
    max_parallel_jobs=2,
    objective_type=objective_type
)

# 튜닝 시작
# tuner.fit({'training': s3_data_uri})

print("하이퍼파라미터 튜닝 설정 완료")

## 9. 정리

학습이 완료되면 다음 작업을 수행하세요:

1. ✓ 학습된 모델 다운로드
2. ✓ 모델 성능 평가
3. ✓ 엔드포인트 삭제 (비용 절감)
4. ✓ S3에 백업된 체크포인트 확인